In [11]:
import os
from typing import Annotated
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.checkpoint.memory import InMemorySaver
from langchain_openai import AzureChatOpenAI

In [5]:
llm = AzureChatOpenAI(
    api_key=os.environ['AZURE_OPENAI_KEY'],
    azure_endpoint=os.environ['AZURE_OPENAI_ENDPOINT'],
    azure_deployment=os.environ['AZURE_OPENAI_DEPLOYMENT_ID'],
    api_version=os.environ['AZURE_OPENAI_API_VERSION'],
    temperature=0,
    timeout=20,
    max_retries=2,
)

# Memory

In [6]:
def call_model(state: MessagesState):
    response = llm.invoke(state["messages"])
    return {"messages": response}

## No STM

In [7]:
builder = StateGraph(MessagesState)
builder.add_node(call_model)
builder.add_edge(START, "call_model")
builder.add_edge("call_model", END)
graph = builder.compile()

In [9]:
for chunk in graph.stream(
    {"messages": [{"role": "user", "content": "hi! I'm bob"}]},
    stream_mode="values",
):
    chunk["messages"][-1].pretty_print()

for chunk in graph.stream(
    {"messages": [{"role": "user", "content": "what's my name?"}]},
    stream_mode="values",
):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

hi! I'm bob


================================== Ai Message ==================================

Hi Bob! How can I assist you today?
================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

I'm sorry, but I don't have access to personal information about users unless it has been shared with me in the course of our conversation. If you'd like, you can tell me your name!


# STM

In [12]:
builder = StateGraph(MessagesState)
builder.add_node(call_model)
builder.add_edge(START, "call_model")
builder.add_edge("call_model", END)
checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [13]:
config = {
    "configurable": {
        "thread_id": "1"
    }
}

In [14]:
for chunk in graph.stream(
    {"messages": [{"role": "user", "content": "hi! I'm bob"}]},
    config,
    stream_mode="values",
):
    chunk["messages"][-1].pretty_print()

for chunk in graph.stream(
    {"messages": [{"role": "user", "content": "what's my name?"}]},
    config,
    stream_mode="values",
):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

hi! I'm bob
================================== Ai Message ==================================

Hi Bob! How can I assist you today?
================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

Your name is Bob! How can I help you today?


# Context Management

- Search Functions:
    - Simple answers: Summarise and return the answer
    - Research Answers: Extract important information and store in VDB, probably need summary of ehat was extraccted also, and a fucntion return this summary.

- RAG:
    Feedback loop on whether it is useful and either extract the useful info or the feedback if it is sufficient

- Task Decomposition:
    Dont store the tasks in the context but im an interactable functions to add tasks, mark tasks as completed, get next tasks etc.


